## Problema dos voos/Flight problem
PT: Seis pessoas precisam sair de diferentes Estados e chegar ao aeroporto de Guarulhos (GRU) em São Paulo. Chegando lá, elas alugarão um carro e dividirão os custos. Para cada pessoa existe dez voos possíveis durante o dia. Dessa forma, qual é a melhor combinação de horários de partida para que os primeiros a chegar não esperem por muito tempo e o custo total seja minimizado?
Em relação ao retorno para os Estados de origem, qual o melhor horário de partida do aeroporto de Guarulhos para que os últimos a partirem esperem o mínimo possível?      
Utilizando os algoritmos a seguir encontramos boas soluções.    
Considerações do problema:       
Penalidade de atraso de devolução do veículo (caso o veículo seja devolvido no dia posterior ao dia do aluguel): aumento de 50 unidades     
Tempo de espera: 1 unidade por minuto    

EN: Six people need to leave different states and arrive at Guarulhos airport (GRU) in São Paulo. Once there, they will rent a car and share the costs. For each person there are ten possible flights during the day. So what is the best combination of departure times so that first-timers don't wait too long and the total cost is minimized? Regarding the return to the States of origin, what is the best time of departure from Guarulhos airport so that the last ones to depart wait as little as possible?     
Using the following algorithms we found good solutions.    
Problem considerations:     
Penalty for late return of the vehicle (if the vehicle is returned the day after the rental day): increase of 50 units    
Standby time: 1 unit per minute     
 

In [1]:
import random
import time
import math

### Organizando os dados do problema.

In [2]:
pessoas=[('Amanda','CWB'), #Criamos uma lista com tuplas contendo (nome, origem) para cada pessoa.
        ('Pedro','GIG'),
         ('Marcos','POA'),
        ('Priscila','FLN'),
        ('Jessy','CNF'),
        ('Paulo','GYN')]

destino='GRU' #Local de destino



In [3]:
voos={} #Criamos um dicionário vazio.
for linha in open('voos.txt'): #criamos um laço de repetição para organizar os voos em dicionário.
    _origem,_destino,_saida,_chegada,_preço=linha.split(',') #criamos variáveis e armazenamos os valores para cada váriável separada por virgula no txt
    voos.setdefault((_origem,_destino),[])#Inserimos a chave (origem,destino no dicionario e uma lista vazia) 
    voos[_origem,_destino].append((_saida,_chegada,int(_preço))) #adicionamos os valores horario de partida, chegada e custo da passagem na chave.
#voos

In [4]:
agenda=[1,3, 5,2, 7,3 , 6,8, 1,4,  0,3 ] 
# As soluções serão em formato de lista em que cada numero (entre 0 e 9) corresponde o voo de ida e volta de uma pessoa
#Exemplo: A primeira pessoa (Amanda) pega o segundo voo de ida e o quarto voo de volta na solução arbitrária acima

In [5]:
#Ordem das pessoas na solução
for i in range(len(pessoas)):
    print(pessoas[i][0],pessoas[i][1])

Amanda CWB
Pedro GIG
Marcos POA
Priscila FLN
Jessy CNF
Paulo GYN


In [6]:
#Criamos uma função para imprimir a solução de maneira agradável
def imprimir_agenda(agenda):
    id_voo=-1
    for i in range(len(agenda)//2):#Como len(agenda)//2=6 pessoas, pegamos o indice da ida e da volta para cada um.
        nome=pessoas[i][0] #Para cada pessoa
        origem=pessoas[i][1] #para cada origem
        id_voo+=1 #incrementamos 1 para ir para o proximo índice, percorrendo a lista da solução
        ida=voos[(origem,destino)][agenda[id_voo]] #vou no dicionario voos, na chave origem e destino, pego o indice de numero que está na solução
        id_voo+=1 #incrimentamos 1 e vamos para o proximo indice da solução
        volta=voos[(destino,origem)][agenda[id_voo]] #pegamos o indice que ocupa a posição atual que corresponde ao voo de volta para a pessoa
        print('%10s%10s %5s-%5s R$%3s %5s-%5s R$%3s'%(nome,origem,ida[0],ida[1],ida[2],volta[0],volta[1],volta[2]))
imprimir_agenda(agenda)

    Amanda       CWB  8:04-10:11 R$ 95 10:33-12:03 R$ 74
     Pedro       GIG 13:54-18:02 R$294  9:49-13:51 R$229
    Marcos       POA 17:08-19:08 R$262 10:32-13:16 R$139
  Priscila       FLN 15:34-18:11 R$326 18:07-21:30 R$355
     Jessy       CNF  8:25-10:34 R$157 12:08-14:47 R$231
     Paulo       GYN  6:11- 8:31 R$249 11:07-13:24 R$171


In [7]:
def get_minutos(hora): #convertendo hora pra minutos
    x=time.strptime(hora,'%H:%M')
    minutos=x[3]*60+x[4]
    return minutos

### Definindo a função custo

In [8]:
def funcao_custo(solucao):
    preco_total=0
    ultima_chegada=0 #em minutos 00:00
    primeira_partida=1439 #primeira partida, na volta, proximo de meia noite, visto que tal horário corresponderia a 0 min.
    id_voo=-1
    for i in range(len(solucao)//2):
        origem=pessoas[i][1] #Pego a origem da cada pessoa
        id_voo+=1 
        ida=voos[(origem, destino)][solucao[id_voo]] #vou no diciario voos(origem de cada, GRU sp) e procuro pelo indice que esta na posição 0
        id_voo+=1
        volta=voos[(destino,origem)][solucao[id_voo]] #pego o preço da volta na chave da solução
        preco_total+=ida[2] #Incrementamos a o valor da ida
        preco_total+=volta[2] #Incrementamos o valor da volta
        if ultima_chegada<get_minutos(ida[1]): #estrutura condicional para controlar a ultima chegada
            ultima_chegada=get_minutos(ida[1]) #ultima pessoa a chegar
        if primeira_partida>get_minutos(volta[0]):#estrutura condicional para controlar a primeira partida, na volta
            primeira_partida=get_minutos(volta[0]) #primeira partida, na volta
    id_voo=-1
    total_espera=0 #espera em minutos. OBS: cada minuto aumenta 1 unidade no custo.
    for i in range(len(solucao)//2): #Laço de repetição para calcular o custo de espera.
        origem=pessoas[i][1]
        id_voo+=1
        ida=voos[(origem, destino)][solucao[id_voo]]
        id_voo+=1
        volta=voos[(destino,origem)][solucao[id_voo]]
        total_espera+=ultima_chegada - get_minutos(ida[1])
        total_espera+=get_minutos(volta[0])-primeira_partida
    if ultima_chegada>primeira_partida: #Penalidade no aluguel do veículo
        preco_total+=50 #Incrementa 50 unidades no preço total
    return preco_total+total_espera #retorna o preço total + preço da espera

In [9]:
print(f'{funcao_custo(agenda)}. Não foi um bom custo') #calculando o custo da solução arbitraria

5245. Não foi um bom custo


### Organizando o domínio do problema

In [10]:
dominio=[(0,9)]*len(pessoas)*2 #uma lista de 12 itens que será utilizada para pesquisar qual voo de ida e volta cada pessoa pegará

### Algoritmo de Pesquisa Randômica
Este algoritmo gera soluções randômicas e escolhe a que possui menor custo, neste caso.

In [17]:
def pesquisa_randomica(dominio,funcao_custo):
    melhor_custo=9999999 #Colocamos um valor alto para logo ser substituido
    for i in range(0,100000): #Gerando 100 000 soluções randômicas
        solucao=[random.randint(dominio[i][0],dominio[i][1]) for i in range(len(dominio))]
        custo=funcao_custo(solucao) #calculando o custo de cada uma
        if custo<melhor_custo: #estrutura condicional para selecionar o menor custo
            melhor_custo=custo 
            melhor_solucao=solucao
   
    return melhor_solucao #retorna a solução em formato de lista
        

In [18]:
solucao_randomica=pesquisa_randomica(dominio,funcao_custo) #Rodando o algoritmo
imprimir_agenda(solucao_randomica) #Imprimindo a solução de menor custo

    Amanda       CWB  9:45-11:50 R$172  9:58-11:18 R$130
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 10:53-13:36 R$189  8:19-11:16 R$122
  Priscila       FLN 11:28-14:40 R$248  6:33- 9:14 R$172
     Jessy       CNF 11:01-12:39 R$260  7:50-10:08 R$164
     Paulo       GYN 12:18-14:56 R$172  8:04-10:59 R$136


In [19]:
custo_solucao_randomica=funcao_custo(solucao_randomica) #calculando o valor da solução encontrada
print(f'Comparação de custos; Custo da solução inicial (agenda): {funcao_custo(agenda)}, Custo da solução randomica: {custo_solucao_randomica}')

Comparação de custos; Custo da solução inicial (agenda): 5245, Custo da solução randomica: 3433


### Algoritmo Subida de Encosta
Este algoritmo encontra os mínimos e máximos locais. Sendo assim utilizarei como solução inicial a solução encontrada na pesquisa randomica "solucao_randomica". Para cada valor nos indices da solução (x), serão geradas duas listas (vizinhos) com o valor alterado da seguinte forma:    
vizinhos=[[x+1, ...],[x-1, ...], [x, y+1,..],[x,y-1,...],...].    
O algoritmo calcula o custo de cada solução e substitui a solucao incial por uma de menor custo e utiliza a nova solução como canditado à minimização, até que um minimo local seja encontrado.

In [20]:
def subida_encosta(dominio, funcao_custo):
    solucao=solucao_randomica
    while True:
        vizinhos=[]
        for i in range(len(dominio)):
            if solucao[i]>dominio[i][0]: #Estrutura de controle para somar ou subtrair 1 e manter o dominio correto
                if solucao[i]!=dominio[i][1]: #se o valor for diferente de 9 somamos 1
                    vizinhos.append(solucao[0:i]+[solucao[i]+1]+solucao[i+1:])
            if solucao[i]<dominio[i][1]:
                if solucao[i]!=dominio[i][0]: #se o valor for diferente de 0 subtraimos 1
                    vizinhos.append(solucao[0:i]+[solucao[i]-1]+solucao[i+1:])
        atual=funcao_custo(solucao)
        melhor=atual
        for i in range(len(vizinhos)):
            custo=funcao_custo(vizinhos[i])
            if custo<melhor:
                melhor=custo
                solucao=vizinhos[i]
        if melhor==atual:
            break
    return solucao
    

In [21]:
solucao_subida_encosta=subida_encosta(dominio,funcao_custo)
imprimir_agenda(solucao_subida_encosta)

    Amanda       CWB 12:34-15:02 R$109  6:39- 8:09 R$ 86
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 12:08-14:59 R$149  8:19-11:16 R$122
  Priscila       FLN 11:28-14:40 R$248  6:33- 9:14 R$172
     Jessy       CNF 12:44-14:17 R$134  7:50-10:08 R$164
     Paulo       GYN 12:18-14:56 R$172  8:04-10:59 R$136


In [26]:
custo_subida_encosta=funcao_custo(solucao_subida_encosta)

In [27]:
print(custo_subida_encosta)

2618


In [24]:
print(f'Custo da solução subida de encosta: {custo_subida_encosta}, pesquisa randomica:{custo_solucao_randomica}, solucao inicial:{funcao_custo(agenda)}')

Custo da solução subida de encosta: 2618, pesquisa randomica:3433, solucao inicial:5245


### Algoritmo de Recozimento Simulado
Algoritmo de busca local probabilística, análogo a processos termodinâmicos.      
[Simulated Annealing](https://pt.wikipedia.org/wiki/Simulated_annealing)     
Pegaremos várias soluções em direções aleatórias a fim de encontrar a melhor solução.

In [34]:
def tempera_simulada(dominio,funcao_custo,temperatura=10000.0,resfriamento=0.95,passo=1):
    solucao=[random.randint(dominio[i][0],dominio[i][1]) for i in range(len(dominio))] #iniciamos uma solucao aleatoria
    while temperatura>0.1:
        i=random.randint(0,len(dominio)-1)
        direcao=random.randint(-passo,passo) #geramos numeros aleatorios: 1 ou -1
        solucao_temp=solucao[:]
        solucao_temp[i]+=direcao #incremento +1 ou decremento 1 no indice i atual
        if solucao_temp[i]<dominio[i][0]: #estrutura de controle do dominio.
             solucao_temp[i]=dominio[i][0]
        elif solucao_temp[i]>dominio[i][1]:
             solucao_temp[i]=dominio[i][1]
        custo_solucao=funcao_custo(solucao)
        custo_solucao_temp=funcao_custo(solucao_temp)
        probabilidade=pow(math.e,(-custo_solucao_temp-custo_solucao)/temperatura) #formula do algoritmo para redução de probabilidade de aceitar soluções inadequadas
        if (custo_solucao_temp<custo_solucao or random.random()<probabilidade): #permitimos que ele pegue soluções um pouco pior do que a atual.
            solucao=solucao_temp
        temperatura=temperatura*resfriamento #faz o resfriamento
    return solucao
    
            

In [43]:
imprimir_agenda(tempera_simulada(dominio,funcao_custo))

    Amanda       CWB 17:11-18:30 R$108  8:23-10:28 R$149
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 17:08-19:08 R$262  8:19-11:16 R$122
  Priscila       FLN 15:34-18:11 R$326 20:27-23:42 R$169
     Jessy       CNF 15:58-18:40 R$173 10:33-13:11 R$132
     Paulo       GYN 16:51-19:09 R$147  8:04-10:59 R$136


In [44]:
funcao_custo(tempera_simulada(dominio,funcao_custo)) #custo da solucao

3969

In [19]:
imprimir_agenda(tempera(dominio,funcao_custo))

    Amanda       CWB 11:16-13:29 R$ 83  6:39- 8:09 R$ 86
     Pedro       GIG  6:12-10:22 R$230 14:20-17:32 R$332
    Marcos       POA 10:53-13:36 R$189  8:19-11:16 R$122
  Priscila       FLN  9:15-12:29 R$225  6:33- 9:14 R$172
     Jessy       CNF  9:42-11:32 R$169  7:50-10:08 R$164
     Paulo       GYN  9:15-12:03 R$ 99  8:04-10:59 R$136


### Algoritmo Genético
Este algoritmo realiza combinação entre soluções ou mutação em um dos índices da solução

In [45]:
def mutacao(dominio,passo,solucao): #função que muta um dos indices da solução de acordo com probabilidades aleatorias
    i=random.randint(0,len(dominio)-1)
    mutante=solucao
    if random.random()<0.5: 
        if solucao[i]!=dominio[i][0]:
            mutante=solucao[0:i]+[solucao[i]-passo]+solucao[i+1:] #Se o indice da solução for diferente de 0, decrementamos 1
    else:
        if solucao[i]!=dominio[i][1]:
            mutante=solucao[0:i]+[solucao[i]+passo]+solucao[i+1:]#Se o indice da solução for diferente de 9, incrementamos 1
    return mutante #retorna solução modificada
        

In [46]:
solucao_exemplo=[3, 0, 2, 7, 3, 1, 2, 0, 2, 1, 2, 1] #Exemplo
mutacao(dominio,1,solucao_exemplo)

[3, 0, 2, 7, 3, 1, 2, 0, 2, 1, 3, 1]

In [47]:
def cruzamento(dominio,solucao1,solucao2): #Combina duas soluções
    i=random.randint(1,len(dominio)-2)
    return solucao1[0:i]+solucao2[i:]


In [48]:
def genetico(dominio,funcao_custo,tamanho_populacao=150,passo=1,probabilidade_mutacao=0.2,elitismo=0.2,geracoes=2000):
    populacao=[]
    for i in range(tamanho_populacao): #iniciamos com uma populaçao de 150 soluções aleatorias
        solucao=[random.randint(dominio[i][0],dominio[i][1]) for i in range(len(dominio))]
        populacao.append(solucao)
    numero_elitismo=int(elitismo*tamanho_populacao) #20% das soluções
    for i in range(geracoes): #testamos 2000 gerações
        custos=[(funcao_custo(individuo),individuo) for individuo in populacao] #criamos uma lista com itens (custo, solução)
        custos.sort()#ordenamos os custos
        individuos_ordenados=[individuo for (custo,individuo) in custos]#pego as soluções ordenadas
        populacao=individuos_ordenados[0:numero_elitismo]#pego 20% das soluções, das melhores neste caso
        
        while len(populacao)<tamanho_populacao: #enquanto os 20% evoluem até chegar ao tamanho 150 individuos
            if random.random()<probabilidade_mutacao: #probabildiade de mutação = 20%
                m=random.randint(0,numero_elitismo) 
                populacao.append(mutacao(dominio,1,individuos_ordenados[m]))#seleciona um individuo entre os 20% e aplica mutação
            else:
                c1=random.randint(0,numero_elitismo)
                c2=random.randint(0,numero_elitismo)
                if c1!=c2: #pegamos duas soluções diferentes entre os 20% e combinamos
                    populacao.append(cruzamento(dominio,individuos_ordenados[c1],individuos_ordenados[c2]))
    return custos[0][1]  #Retornar a solução de menor custo                 
        

In [49]:
solucao_algoritmo_genetico=genetico(dominio,funcao_custo)
solucao_algoritmo_genetico

[4, 3, 3, 3, 4, 3, 3, 4, 4, 3, 4, 3]

In [53]:
print(f'{funcao_custo(solucao_algoritmo_genetico)}. A melhor solução até o momento!!!')

2356. A melhor solução até o momento!!!


In [54]:
imprimir_agenda(solucao_algoritmo_genetico)

    Amanda       CWB 12:34-15:02 R$109 10:33-12:03 R$ 74
     Pedro       GIG 10:30-14:57 R$290 10:51-14:16 R$256
    Marcos       POA 12:08-14:59 R$149 10:32-13:16 R$139
  Priscila       FLN 11:28-14:40 R$248 12:37-15:05 R$170
     Jessy       CNF 12:44-14:17 R$134 10:33-13:11 R$132
     Paulo       GYN 12:18-14:56 R$172 11:07-13:24 R$171
